In [25]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
import re
import sys
import time

import numpy as np
import pandas as pd
import os
import psycopg2

from tqdm import tqdm

In [26]:
try:
    skiplist = list(pd.read_csv('skiplist.csv',index_col=0,header=None).iloc[:,0])
except:
    skiplist = []

In [27]:
skiplist

[115922,
 119509,
 119510,
 18780,
 25273,
 28631,
 29112,
 31395,
 32583,
 33032,
 33276,
 115922,
 119509,
 119510,
 25273,
 115922,
 119509,
 119510,
 25273,
 67583,
 68459,
 80854,
 100,
 1000,
 100000,
 100024,
 10008,
 100082,
 100086,
 100088,
 100092,
 100094,
 100098,
 100100,
 100122,
 100143,
 100151,
 100153,
 100160,
 116398,
 10018,
 100186,
 100192,
 100195,
 100209,
 100221,
 100222,
 100230,
 100236,
 104692,
 100245,
 100247,
 10026,
 100263,
 100271,
 100274,
 100279,
 100281,
 100297,
 10031,
 100333,
 100334,
 100337,
 10035,
 100358,
 10036,
 100367,
 100369,
 10037,
 100371,
 100387,
 100388,
 100390,
 100418,
 101909,
 102057,
 102058,
 102059,
 10206,
 102060,
 102061,
 102062,
 102063,
 102064,
 102065,
 102066,
 102067,
 102068,
 102069,
 102070,
 102071,
 102072,
 102073,
 102074,
 102075,
 102076,
 102077,
 102078,
 102079,
 10208,
 102080,
 102081,
 102082,
 102083,
 102084,
 102085,
 102086,
 102087,
 102088,
 102089,
 10209,
 102090,
 102091,
 102092,
 1

In [28]:
conn = psycopg2.connect(host="chinaquant.cofdbub8go3o.eu-west-1.rds.amazonaws.com",
                        database="postgres",
                        user="tokyo3",
                        password="evangelion")

cur = conn.cursor()

In [29]:
command = """
SELECT doc_id,doc_url
FROM documents
WHERE doc_html IS NULL AND doc_date IS NOT NULL and doc_id NOT IN {}
LIMIT 5
""".format(tuple(skiplist))

cur.execute(command)

existing = cur.fetchall()

In [30]:
existing

[(10340,
  'https://www.fmprc.gov.cn/web/gjhdq_676201/gj_676203/yz_676205/1206_676207/1206x2_676229/t1586159.shtml'),
 (103400,
  'https://www.fmprc.gov.cn/web/gjhdq_676201/gj_676203/dyz_681240/1206_681242/xgxw_681248/t359240.shtml'),
 (103401,
  'https://www.fmprc.gov.cn/web/gjhdq_676201/gj_676203/dyz_681240/1206_681242/xgxw_681248/t359073.shtml'),
 (103402,
  'https://www.fmprc.gov.cn/web/gjhdq_676201/gj_676203/dyz_681240/1206_681242/xgxw_681248/t358843.shtml'),
 (103403,
  'https://www.fmprc.gov.cn/web/gjhdq_676201/gj_676203/dyz_681240/1206_681242/xgxw_681248/t358299.shtml')]

In [31]:
conn = psycopg2.connect(host="chinaquant.cofdbub8go3o.eu-west-1.rds.amazonaws.com",
                    database="postgres",
                    user="tokyo3",
                    password="evangelion")

cur = conn.cursor()

command = """
SELECT count(doc_id)
FROM documents
WHERE doc_html IS NULL AND doc_date IS NOT NULL and doc_id NOT IN {}

""".format(tuple(skiplist))

cur.execute(command)

existing = cur.fetchall()

In [32]:
existing

[(109499,)]

In [33]:
command = """
SELECT column_name
FROM information_schema.columns
WHERE table_name = 'documents'
"""

cur.execute(command)

columns = cur.fetchall()

In [34]:
columns

[('doc_id',),
 ('doc_country',),
 ('doc_language',),
 ('doc_page_name',),
 ('doc_url',),
 ('doc_nav',),
 ('doc_title',),
 ('doc_date',),
 ('doc_html',),
 ('doc_content',)]

In [35]:
def html(url):
    try:
        driver.get(url)
        html = driver.page_source
        if type(html) == str:
            return(html)
        else:
            return('ERROR')
    except:
        return('ERROR')

In [36]:
def headlessDriver():
    options = Options()
    options.add_argument("--headless")
    driver = webdriver.Firefox(options=options, executable_path="geckodriver.exe")
    return(driver)

In [37]:
def buildEntry(index_value,url):
    t_start = time.time()
    
    index = index_value
    pageSource = html(url)
#     pageDate = str()
    pageText = str()
    if pageSource == 'ERROR':
#         pageDate = 'ERROR'
        pageText = 'ERROR'
    else:
        try:
            soup=BeautifulSoup(pageSource, 'lxml')
#             pageDate = soup.find("span", {"id": "News_Body_Time"}).text
#             if pageDate == '':
#                 pageDate = None
            pageText = soup.find(class_='content').text
            if pageText == '':
                pageText = None
        except:
#             pageDate = 'ERROR'
            pageText = 'ERROR'
    
    entry = (pageSource,
#              pageDate,
             pageText,
             index)
    
    t_end = time.time()
    
    duration = t_end - t_start
    
    if duration >= 10:
        time.sleep(np.random.randint(5,15))
    
#   print(duration)          
    return(entry)

In [38]:
conn = psycopg2.connect(host="chinaquant.cofdbub8go3o.eu-west-1.rds.amazonaws.com",
                    database="postgres",
                    user="tokyo3",
                    password="evangelion")

cur = conn.cursor()

command = """
SELECT doc_id,doc_url
FROM documents
WHERE doc_html IS NULL AND doc_date IS NOT NULL and doc_id NOT IN {}
LIMIT 10
""".format(tuple(skiplist))

cur.execute(command)

existing = cur.fetchall()

cur.close()
conn.close()

In [39]:
def fullHarvest():
    t = time.time()
    try:
        skiplist = list(pd.read_csv('skiplist.csv',index_col=0,header=None).iloc[:,0])
    except:
        skiplist = []
        
    
    conn = psycopg2.connect(host="chinaquant.cofdbub8go3o.eu-west-1.rds.amazonaws.com",
                        database="postgres",
                        user="tokyo3",
                        password="evangelion")

    cur = conn.cursor()
    
    command = """
    SELECT doc_id,doc_url
    FROM documents
    WHERE doc_html IS NULL AND doc_date IS NOT NULL and doc_id NOT IN {}
    LIMIT 10
    """.format(tuple(skiplist))

    cur.execute(command)

    existing = cur.fetchall()
#     print(existing)
    cur.close()
    conn.close()
    
#     driver = None
#     driver = headlessDriver()
    submission = [buildEntry(int(x[0]),str(x[1])) for x in existing]
    
    newSubmission = []
    for i in submission:
        if i[1] == 'ERROR':
            skiplist.append(i[2])
        else:
            newSubmission.append(i)
    
    
    pd.Series(skiplist).to_csv('skiplist.csv')
    
    
    
    conn = psycopg2.connect(host="chinaquant.cofdbub8go3o.eu-west-1.rds.amazonaws.com",
                        database="postgres",
                        user="tokyo3",
                        password="evangelion")

    cur = conn.cursor()
    sql = """
        UPDATE documents
        SET doc_html = %s,doc_content = %s
        WHERE doc_id = %s;
        """

    cur.executemany(sql,newSubmission)
    conn.commit()
    
    
    command = """
    SELECT COUNT(doc_id)
    FROM documents
    WHERE doc_html IS NULL AND doc_date IS NOT NULL and doc_id NOT IN {}
    """.format(tuple(skiplist))


    cur.execute(command)

    count = cur.fetchall()
    

    cur.close()
    conn.close()
    
    nt = time.time()
    dt = nt - t
    
    return(len(skiplist),count,dt)

    

In [40]:
for i in tqdm(range(100)):
    driver = headlessDriver()
    s,r,t = fullHarvest()
    print("{} : {:.4f}s : {} skips : {} remaining".format(i,t,s,r))


  0%|          | 0/100 [00:00<?, ?it/s]

0 : 28.0256s : 327 skips : [(109489,)] remaining



  1%|          | 1/100 [00:32<54:00, 32.73s/it]

1 : 29.3570s : 327 skips : [(109479,)] remaining



  2%|▏         | 2/100 [01:06<53:53, 32.99s/it]

...

20 : 10.2880s : 513 skips : [(109289,)] remaining



 21%|██        | 21/100 [06:12<20:36, 15.65s/it]

21 : 38.7773s : 513 skips : [(109279,)] remaining



 22%|██▏       | 22/100 [06:56<31:12, 24.01s/it]

WebDriverException: Message: Process unexpectedly closed with status 1


In [ ]:
# driver = headlessDriver()
# buildEntry(102358,"""https://www.fmprc.gov.cn/web/gjhdq_676201/gj_676203/nmz_680924/1206_681192/1206x2_681212/t878484.shtml""")

In [ ]:
try:
    skiplist = list(pd.read_csv('skiplist.csv',index_col=0,header=None).iloc[:,0])
except:
    skiplist = []

In [ ]:
skiplist

In [ ]:


conn = psycopg2.connect(host="chinaquant.cofdbub8go3o.eu-west-1.rds.amazonaws.com",
                        database="postgres",
                        user="tokyo3",
                        password="evangelion")

cur = conn.cursor()

command = """
SELECT COUNT(doc_id)
FROM documents
WHERE doc_html IS NULL AND doc_date IS NOT NULL and doc_id NOT IN {}
""".format(tuple(skiplist))


cur.execute(command)

extract = cur.fetchall()
print(extract)
cur.close()
conn.close()

In [ ]:
# submission[-1]

In [ ]:
# for i in submission:
#     print(i[2],'\n')

In [ ]:
# print(len(submission))

In [ ]:
# for i in submission:
#     if i[2] == 'ERROR':
#         print(i[3])

In [ ]:
# newSubmission = []
# for i in submission:
#     if i[1] == 'ERROR':
#         skiplist.append(i[3])
#     else:
#         newSubmission.append(i)

In [ ]:
# pd.Series(skiplist).to_csv('skiplist.csv')

In [ ]:
# pd.Series(skiplist)

In [ ]:
# conn = psycopg2.connect(host="chinaquant.cofdbub8go3o.eu-west-1.rds.amazonaws.com",
#                         database="postgres",
#                         user="tokyo3",
#                         password="evangelion")

# cur = conn.cursor()

In [ ]:
# sql = """
#         UPDATE documents
#         SET doc_html = %s,doc_date = %s,doc_content = %s
#         WHERE doc_id = %s;
#         """

# cur.executemany(sql,newSubmission)
# conn.commit()

# cur.close()
# conn.close()

In [ ]:
# existing

In [ ]:
# len(existing)

In [ ]:
# listCheck = [x[0] for x in existing]

In [ ]:
# listCheck

In [ ]:
# tuple(listCheck)

In [ ]:
# conn = psycopg2.connect(host="chinaquant.cofdbub8go3o.eu-west-1.rds.amazonaws.com",
#                         database="postgres",
#                         user="tokyo3",
#                         password="evangelion")

# cur = conn.cursor()

# command = """
# SELECT *
# FROM documents
# WHERE doc_id IN {}
# """.format(tuple(listCheck))

# cur.execute(command)

# extract = cur.fetchall()

# cur.close()
# conn.close()

In [ ]:
# conn = psycopg2.connect(host="chinaquant.cofdbub8go3o.eu-west-1.rds.amazonaws.com",
#                         database="postgres",
#                         user="tokyo3",
#                         password="evangelion")

# cur = conn.cursor()

# command = """
# SELECT COUNT(doc_id)
# FROM documents
# WHERE doc_html IS NULL AND doc_date IS NULL and doc_id NOT IN {}
# """.format(tuple(skiplist))


# cur.execute(command)

# extract = cur.fetchall()

# cur.close()
# conn.close()

In [ ]:
# extract